In [1]:
# coding: utf-8

import numpy as np
import torch
import os
import time

import sys
sys.path.append('/home/tudou/Github/tmp/MobilePose')

# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline  

from networks import *
from network import CoordRegressionNetwork
from torch.utils.data import DataLoader
from dataset_factory import DatasetFactory
import multiprocessing
from default import get_cfg_defaults

# gpu setting
os.environ["CUDA_VISIBLE_DEVICES"]="1"
torch.backends.cudnn.enabled = True
device = torch.device("cuda" if True else "cpu")
num_threads = multiprocessing.cpu_count()
print("GPU NUM: %d, Thread NUM: %d"%(torch.cuda.device_count(),num_threads))

%load_ext autoreload
%autoreload 2

GPU NUM: 1, Thread NUM: 48


In [11]:
config_file = '/home/tudou/Github/tmp/MobilePose/config/coord_resnet18.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model_path = "/home/tudou/Github/tmp/MobilePose/experiment/weight/opt-phone-resnet18-320-8kp-best.pth"
# model_path = "experiment/weight2/opt-phone-resnet18-320-8kp-best.pth"
net = CoordRegressionNetwork(n_locations=cfg.TASK.COORD_NUM, backbone=cfg.MODEL.BACKBONE.NAME)
net.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
net = net.eval()

# inputsize = 480
# x = torch.randn((2,3,480,480))
# y = net(x)
# print(y[0].shape)

In [12]:
import cv2

def rescale(image, output_size):
    image = image / 255
    h, w = image.shape[:2]
    # if h == 0 or w == 0:
    #     print()
    # print(image_.shape)
    im_scale = min(float(output_size[0]) / float(h), float(output_size[1]) / float(w))
    new_h = int(image.shape[0] * im_scale)
    new_w = int(image.shape[1] * im_scale)
    image = cv2.resize(image, (new_w, new_h),
                interpolation=cv2.INTER_LINEAR)
    left_pad = (output_size[1] - new_w) // 2
    right_pad = (output_size[1] - new_w) - left_pad
    top_pad = (output_size[0] - new_h) // 2
    bottom_pad = (output_size[0] - new_h) - top_pad
    mean=np.array([0.485, 0.456, 0.406])
    pad = ((top_pad, bottom_pad), (left_pad, right_pad))
    image = np.stack([np.pad(image[:,:,c], pad, mode='constant', constant_values=mean[c]) 
                    for c in range(3)], axis=2)
    
    return image


def totensor(image):

		# todo: support heatmap
        # guass_heatmap = sample['guass_heatmap']
        h, w = image.shape[:2]

        mean=np.array([0.485, 0.456, 0.406])
        std=np.array([0.229, 0.224, 0.225])

        image[:,:,:3] = (image[:,:,:3]-mean)/(std)
        image = image.transpose((2, 0, 1))
        image = image[np.newaxis, ...]
        image = torch.from_numpy(image).float()

		# todo: support heatmap
	    # guass_heatmap = torch.from_numpy(guass_heatmap).float()
        return image

In [13]:
image = cv2.imread("keypoint.jpeg")
image = rescale(image, [480, 480])

# image = cv2.imread("/data3/tudou/data/keypoint/opt-for-detect/Image_20211217171609611.jpeg")
# image = rescale(image, [320, 320])
image = totensor(image)
print(image.shape)

torch.Size([1, 3, 480, 480])


In [14]:

from dataloader import display_pose

coords, heatmaps = net(image)

drawed = display_pose(image[0][:3,:,:], coords[0], 0)
cv2.imwrite("output.jpg", drawed)


True